In [42]:
import cv2

# Load an image from file
image = cv2.imread("/Users/abdurrehman/C++Prac/Zero-mean-Normalized-Cross-Correlation-Algorithm/images_smallest/file0.png", cv2.IMREAD_GRAYSCALE)
# cv2.imshow("Image", image)
# # Wait for a key press and close the window when any key is pressed
# cv2.waitKey(0)
# cv2.destroyAllWindows()


def template_image(image, window_size):
    averaged_image = image; 
    for j in range(len(image) - window_size + 1):
        for i in range(len(image[0]) - window_size + 1):
            window_average = 0
            for win_y in range(window_size):
                for win_x in range(window_size):
                    window_average += image[win_y + j][win_x + i]
            window_average /= window_size * window_size
            for win_y in range(window_size):
                    for win_x in range(window_size):
                        if(win_x == 1 and win_y == 1):
                            averaged_image[win_y + j][win_x + i] -= window_average
    return averaged_image


final_image1 = template_image(image, 3)
# cv2.imshow("Image", final_image)
# # Wait for a key press and close the window when any key is pressed
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# print(final_image)

In [46]:
image = cv2.imread("/Users/abdurrehman/C++Prac/Zero-mean-Normalized-Cross-Correlation-Algorithm/images_smallest/file1.png", cv2.IMREAD_GRAYSCALE)

def target_image(image, window_size, disparity):
    averaged_image = image; 
    for j in range(len(image) - window_size + 1):
        for i in range(len(image[0]) - window_size + 1):
            window_average = 0
            for d in range(disparity):
                for win_y in range(window_size):
                    for win_x in range(window_size):
                        window_average += image[win_y + j][win_x + i - d]
                window_average /= window_size * window_size
                for win_y in range(window_size):
                        for win_x in range(window_size):
                            if(win_x == 1 and win_y == 1):
                                averaged_image[win_y + j][win_x + i - d] -= window_average
    return averaged_image

final_image2 = target_image(image, 3, 1)
# cv2.imshow("Image", final_image)
# # Wait for a key press and close the window when any key is pressed
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# final_image

new_image

array([[216, 152, 152, ..., 246,  24,  96],
       [220,   0, 143, ..., 146, 144, 205],
       [152,  22, 248, ..., 202,  64,  60],
       ...,
       [164,  64, 145, ..., 136, 146,  53],
       [ 12,   9, 254, ...,  15, 143,  92],
       [131, 132,  16, ..., 141, 174, 224]], dtype=uint8)

In [47]:
cv2.imshow("Image", new_image)
# Wait for a key press and close the window when any key is pressed
cv2.waitKey(0)
cv2.destroyAllWindows()

In [71]:
## THIS IS CHATGPT - GEMINI THINKING OF ZNCC WHEN I SHOWED THEM MY FUNCTIONS..


import math
import numpy as np
import cv2

def calculate_zncc(template_image, target_image, window_size, disparity):
    result_image = []  # Initialize an empty list to store the resulting disparity map

    # Iterate over the valid pixels of the target image (considering window size)
    for j in range(len(target_image) - window_size + 1):
        row = []  # Initialize an empty list to store disparity values for a row
        for i in range(len(target_image[0]) - window_size + 1):
            # Calculate template mean and standard deviation
            template_mean = 0
            template_std = 0
            for win_y in range(window_size):
                for win_x in range(window_size):
                    template_mean += template_image[win_y + j][win_x + i]
            template_mean /= window_size * window_size
            for win_y in range(window_size):
                for win_x in range(window_size):
                    template_std += (template_image[win_y + j][win_x + i] - template_mean)**2
            template_std = math.sqrt(template_std / (window_size * window_size - 1))  # Bessel's correction

            # Iterate over possible disparities
            max_zncc = float('-inf')
            best_disparity = 0
            for d in range(disparity):
                window_mean = 0
                window_std = 0
                for win_y in range(window_size):
                    for win_x in range(window_size):
                        target_pixel = target_image[win_y + j][win_x + i - d]
                        window_mean += target_pixel
                        window_std += (target_pixel - window_mean)**2
                window_mean /= window_size * window_size
                window_std = math.sqrt(window_std / (window_size * window_size - 1))  # Bessel's correction

                # Compute ZNCC value (avoid division by zero)
                zncc_value = 0
                if template_std * window_std > 0:
                    for win_y in range(window_size):
                        for win_x in range(window_size):
                            zncc_value += (template_image[win_y + j][win_x + i] - template_mean) * (target_image[win_y + j][win_x + i - d] - window_mean)
                    zncc_value /= (template_std * window_std)

                # Update maximum ZNCC value and best disparity
                if zncc_value > max_zncc:
                    max_zncc = zncc_value
                    best_disparity = d

            # Append the best disparity value to the row
            row.append(best_disparity)
        # Append the row to the result image
        result_image.append(row)

    return result_image

# Example usage with actual images:
resulting_disparity_map = calculate_zncc(final_image1, final_image2, window_size=3, disparity=3)

# Convert the disparity map to a numpy array
resulting_disparity_map = np.array(resulting_disparity_map)

# Normalize the disparity map for visualization
scaled_disparity = np.uint8(255 * (resulting_disparity_map + 1) / (16 + 1))

# Convert the numpy array to a format that cv2.imshow() can handle
disparity_image = cv2.cvtColor(scaled_disparity, cv2.COLOR_GRAY2BGR)

# Display the disparity map
cv2.imshow("Disparity Map", disparity_image)

# Wait for a key press and close the window when any key is pressed
cv2.waitKey(0)
cv2.destroyAllWindows()

resulting_disparity_map


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])